In [ ]:

import pandas as pd
from clickhouse_driver import Client

CH_HOST = 'localhost'
CH_PORT = 9000
CH_USER = 'default'
CH_PASSWORD = ''
CH_DATABASE = 'default'
CH_TABLE = 'e_otinish_clean'

csv_file = 'e_otinish_dummy.csv'
df = pd.read_csv(csv_file, encoding='utf-8-sig')

df = df.rename(columns={
    'ID': 'id',
    'Дата обращения': 'request_date',
    'Категория': 'category',
    'Подкатегория': 'subcategory',
    'Регион': 'region',
    'Возраст заявителя': 'applicant_age',
    'Пол': 'gender',
    'Статус': 'status',
    'Дата ответа': 'response_date',
    'Кол-во детей': 'children_count',
    'Возраст старшего ребенка': 'oldest_child_age',
    'Ответственный орган': 'responsible_agency',
    'Тип заявителя': 'applicant_type',
    'Способ подачи': 'submission_method',
    'Язык обращения': 'language',
    'Повторное обращение': 'is_repeat',
    'Канал получения ответа': 'response_channel',
    'Категория срочности': 'urgency',
    'Вид обращения': 'appeal_type',
    'Тематика': 'topic'
})

df['request_date'] = pd.to_datetime(df['request_date']).dt.date
df['response_date'] = pd.to_datetime(df['response_date']).dt.date
df['applicant_age'] = df['applicant_age'].astype('UInt8')
df['children_count'] = df['children_count'].fillna(0).astype('UInt8')
df['oldest_child_age'] = df['oldest_child_age'].fillna(0).astype('UInt8')

client = Client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD, database=CH_DATABASE)
client.execute(f"DROP TABLE IF EXISTS {CH_TABLE}")
print(f"🗑 Таблица '{CH_TABLE}' удалена (если существовала).")

def create_ch_table():
    client.execute(f"""
        CREATE TABLE {CH_TABLE} (
            id UInt32,
            request_date Date,
            category String,
            subcategory String,
            region String,
            applicant_age UInt8,
            gender String,
            status String,
            response_date Date,
            children_count UInt8,
            oldest_child_age UInt8,
            responsible_agency String,
            applicant_type String,
            submission_method String,
            language String,
            is_repeat String,
            response_channel String,
            urgency String,
            appeal_type String,
            topic String
        ) ENGINE = MergeTree()
        ORDER BY id
    """)

def upload_to_ch():
    data = [tuple(x) for x in df.to_records(index=False)]
    client.execute(f"INSERT INTO {CH_TABLE} VALUES", data)
    print(f"✅ Загружено {len(data)} строк в таблицу '{CH_TABLE}'.")

def connect_to_superset():
    print(f"""
Теперь подключите таблицу в Superset:

1. Перейдите в: Data → Datasets
2. Нажмите: + Dataset
3. Выберите источник ClickHouse
4. База данных: {CH_DATABASE}
5. Таблица: {CH_TABLE}
6. Нажмите Save — и можно строить визуализации!
    """)

create_ch_table()
upload_to_ch()
connect_to_superset()


🗑 Таблица 'e_otinish_clean' удалена (если существовала).
✅ Загружено 10000 строк в таблицу 'e_otinish_clean'.

Теперь подключите таблицу в Superset:

1. Перейдите в: Data → Datasets
2. Нажмите: + Dataset
3. Выберите источник ClickHouse
4. База данных: default
5. Таблица: e_otinish_clean
6. Нажмите Save — и можно строить визуализации!
    
